# Fetch Profile from DB

In [ ]:
!pip install python-dotenv pandas mysql-connector-python ipython


In [ ]:
import os
db_port = os.getenv("DB_PORT", "3306")  # Default to 3306 if missing
print(f"🔍 DEBUG: Raw DB_PORT from .env = '{db_port}'' (Type: {type(db_port)})")

In [2]:
import sys
import os
import textwrap
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display

# Fix for Jupyter: Use os.getcwd() instead of __file__
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

# Load environment variables
load_dotenv()

from src.database.queries import (
    get_profile_by_username,
    get_all_profiles,
    search_profiles,
    get_profile_stats,
    get_posts_with_comments
)

def display_profile(profile):
    """Display profile information and posts using pandas DataFrame in Jupyter Notebook."""
    if not profile:
        print("\nProfile not found.\n")
        return

    print(f"\n📌 Instagram Profile: {profile['username']}\n")

    profile_data = {
        "Attribute": ["Username", "Full Name", "Followers", "Following", "Posts", 
                      "Verified", "Account Type", "Private"],
        "Value": [profile["username"], profile["full_name"], f"{profile['follower_count']:,}",
                  f"{profile['following_count']:,}", f"{profile['media_count']:,}",
                  "✓" if profile["is_verified"] else "✗", profile["account_type"],
                  "Yes" if profile["is_private"] else "No"]
    }
    display(pd.DataFrame(profile_data))

    if profile["biography"]:
        print("\n📜 Bio:")
        print(profile["biography"])

    if profile["bio_links"]:
        bio_links_data = [{"Title": link["title"] or "Link", "URL": link["url"]} for link in profile["bio_links"]]
        print("\n🔗 Bio Links:")
        display(pd.DataFrame(bio_links_data))

    posts = get_posts_with_comments(profile["username"], limit=12)
    if posts:
        print("\n🖼️ Recent Posts:")
        post_data = []
        for post in posts:
            post_entry = {
                "Post ID": post["id"],
                "Likes": post["like_count"],
                "Comments": post["comment_count"],
                "Caption": "\n".join(textwrap.wrap(post["caption"], width=50)) if post["caption"] else "[No caption]",
                "Timestamp": post["timestamp"]
            }
            post_data.append(post_entry)
            
            if post["comments"]:
                comments_data = [{
                    "Post ID": post["id"],
                    "Caption": post["caption"] if post["caption"] else "[No caption]",
                    "Username": c["username"],
                    "Comment": textwrap.fill(c["comment"], width=60)
                } for c in post["comments"]]
                print("\n💬 Comments:")
                display(pd.DataFrame(comments_data))
        
        display(pd.DataFrame(post_data))

def fetch_all_profiles():
    """Fetch all profiles and display using pandas DataFrame."""
    profiles = get_all_profiles(limit=None)
    if not profiles:
        print("No profiles found.")
        return

    profile_data = [
        {"Username": p["username"], "Followers": p["follower_count"], 
         "Following": p["following_count"], "Posts": p["media_count"]}
        for p in profiles
    ]
    print("\n🌍 All Profiles:")
    display(pd.DataFrame(profile_data))

def fetch_statistics():
    """Fetch overall statistics and display using pandas DataFrame."""
    stats = get_profile_stats()
    stats_data = [
        {"Metric": "Total Profiles", "Value": stats.get("total_profiles", 0)},
        {"Metric": "Average Followers", "Value": int(stats.get("avg_followers", 0))},
        {"Metric": "Average Following", "Value": int(stats.get("avg_following", 0))},
        {"Metric": "Verified Accounts", "Value": stats.get("verified_count", 0)},
        {"Metric": "Business Accounts", "Value": stats.get("business_count", 0)},
        {"Metric": "Most Followed", 
         "Value": f"{stats.get('max_followers_username', 'N/A')} ({stats.get('max_followers_count', 0):,} followers)"}
    ]
    print("\n📊 Instagram Statistics:")
    display(pd.DataFrame(stats_data))

# Example Execution in Jupyter Notebook
print("\n1️⃣ Fetching Specific Profiles:")
for username in ["taylorswift", "theweeknd", "sabrinacarpenter"]:
    profile = get_profile_by_username(username)
    display_profile(profile)

print("\n2️⃣ Fetching All Profiles:")
fetch_all_profiles()

print("\n3️⃣ Fetching Overall Statistics:")
fetch_statistics()


1️⃣ Fetching Specific Profiles:
Connected to MySQL database

📌 Instagram Profile: taylorswift



,Attribute,Value
0,Username,taylorswift
1,Full Name,Taylor Swift
2,Followers,"282,078,231"
3,Following,0
4,Posts,676
5,Verified,✓
6,Account Type,Personal
7,Private,No



📜 Bio:
All’s fair in love and poetry... New album THE TORTURED POETS DEPARTMENT. Out now 🤍

🔗 Bio Links:


,Title,URL
0,Link,http://taylorswift.com
1,THE TORTURED POETS DEPARTMENT 🤍,https://taylor.lnk.to/thetorturedpoetsdepartment
2,Taylor Swift | The Eras Tour concert film,https://taylor.lnk.to/TSTheErasTourFilm


Connected to MySQL database

🖼️ Recent Posts:

💬 Comments:


...

,Post ID,Likes,Comments,Caption,Timestamp
0,3344626839165085703,2282489,11968,ESPRESSO VIDEO IS OUT NOW💋 directed by the\ni...,None
1,3382402499083626613,4324267,36216,“Short n’ Sweet” MY NEW ALBUM IS COMING OUT\n...,None
2,3384789075884248360,3640583,27740,PLEASE PLEASE PLEASE OUT NOW 💋 Directed by th...,None
3,3571583779669472319,2334583,6721,just 2 weeks away from the start of europe tou...,None
4,3575920034166647410,1569968,2780,Pradascope🤍 new lash lengthening mascara from\...,None
5,3577309673964220454,1712560,16260,you asked and we listened!!! So excited to\na...,None
6,3579545603907818295,4126175,8460,Brits 🇬🇧💋 i now know what watershed is!!!!,None
7,3580232172902509730,2081425,4446,the British are coming!!! bed chem x espresso ...,None
8,3581747836862510869,3211287,5016,Ireland!!!! No better way to kick off this leg...,None
9,3585231507117477287,3264348,5145,"London, the last two nights at the @theo2londo...",None



2️⃣ Fetching All Profiles:
Connected to MySQL database

🌍 All Profiles:


,Username,Followers,Following,Posts
0,taylorswift,282078231,0,676
1,theweeknd,77166604,7,986
2,sabrinacarpenter,45526594,581,365



3️⃣ Fetching Overall Statistics:
Connected to MySQL database

📊 Instagram Statistics:


,Metric,Value
0,Total Profiles,3
1,Average Followers,134923809
2,Average Following,196
3,Verified Accounts,3
4,Business Accounts,1
5,Most Followed,"taylorswift (282,078,231 followers)"
